# Notebook de Inicialização - Spotify Analytics Data Lakehouse

## Propósito
Este notebook realiza a configuração inicial do ambiente de Data Lakehouse, criando a estrutura de diretórios e schemas necessários para o projeto Spotify Analytics.

## O que este notebook faz:
1. Cria estrutura de diretórios locais para organização dos notebooks
2. Cria o catálogo `spotify_analytics` no Databricks
3. Cria os schemas das camadas da Medallion Architecture:
   - **landing**: Arquivos raw (JSON brutos do Spotify)
   - **bronze**: Dados brutos ingeridos com metadata
   - **silver**: Dados limpos e validados
   - **gold**: Agregações para analytics e BI (inclui views da camada diamond)

4. Cria volume `raw_data` para upload de arquivos JSON

**Observação importante sobre a camada Diamond:**
- A camada Diamond **NÃO possui schema próprio**
- As views da Diamond são criadas dentro do schema `gold`
- Exemplo: `spotify_analytics.gold.vw_track_analytics`
- Diamond é composta apenas de views (não há tabelas físicas)

## Quando executar:
- **UMA VEZ** na criação inicial do projeto
- Sempre que resetar o ambiente completo

## Próximo passo:
Após executar este notebook:
1. Faça upload dos arquivos JSON em `/Volumes/spotify_analytics/landing/raw_data/`
2. Execute os notebooks de DDL para criar as tabelas (bronze, silver, gold)
3. Execute os notebooks de carga na ordem: bronze → silver → gold → diamond

## 1. Criação da Estrutura de Diretórios

In [0]:
import os

base_path = os.getcwd()

dirs = {
    "src": ["landing", "bronze", "silver", "gold", "diamond"],
    "ddl": ["bronze", "silver", "gold"]
}

for parent, subs in dirs.items():
    for sub in subs:
        full_path = os.path.join(base_path, parent, sub)
        os.makedirs(full_path, exist_ok=True)
        print(f"Diretório criado: {full_path}")

## 2. Criação do Catálogo e Schemas

In [0]:
catalog_name = "spotify_analytics"
schemas = ["landing", "bronze", "silver", "gold"]

# Cria o catálogo
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"USE CATALOG {catalog_name}")
print(f"✓ Catálogo criado e selecionado: {catalog_name}")

# Cria os schemas
for schema in schemas:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")
    print(f"✓ Schema garantido: {catalog_name}.{schema}")

# Cria volume para armazenar arquivos raw JSON na camada landing
spark.sql(f"""
    CREATE VOLUME IF NOT EXISTS {catalog_name}.landing.raw_data
    COMMENT 'Volume para armazenar arquivos JSON brutos do Spotify'
""")
print(f"✓ Volume criado: {catalog_name}.landing.raw_data")
print(f"\n📁 Caminho para upload: /Volumes/{catalog_name}/landing/raw_data/")

## 3. Verificação do Ambiente

In [ ]:
# Lista todos os schemas criados
print("\n=== Schemas Criados ===")
spark.sql("SHOW SCHEMAS").show(truncate=False)

print("\n✅ Inicialização concluída com sucesso!")
print("\n📋 Próximos passos:")
print("   1. Execute os DDLs de criação de tabelas em ddl/bronze/")
print("   2. Execute os DDLs de criação de tabelas em ddl/silver/")
print("   3. Execute os DDLs de criação de tabelas em ddl/gold/")
print("   4. Execute os notebooks de carga em src/ na ordem: bronze → silver → gold → diamond")
print("\n💡 Lembre-se: A camada diamond não tem DDLs (são apenas views criadas nos notebooks src/diamond/)")